In [16]:
import numpy
from scholarly import scholarly
import tweepy
import pandas as pd
import numpy as np
from IPython.display import display
import time

In [43]:
def initAPI():
        # Authenticate to Twitter
    auth = tweepy.OAuthHandler("Nx8QQYr0HIvf5LsQBbmOEHkwl", 
        "gq451Xsq5keBqNSoiu2rUY3mPeI0Fqzzrj21UmGWTwzRr8fGeo")
    auth.set_access_token("1237156440713736192-TvUNXIxeY1oX3VukiUBNATFmmgCLJO", 
        "qhj2UV74OKJExhc831HsZeFvRH6cehC54yi0Rr7MnUkxz")

    api = tweepy.API(auth)

    try:
        api.verify_credentials()
        print("Authentication OK")
        return(api)
    except:
        print("Error during authentication")
        return(None)

def search_name(name):
    data = pd.DataFrame(columns=['name','publication'])
    search_query = scholarly.search_author(name)
    author = next(search_query).fill()
    for pub in author.publications:
        data = data.append({'name':name, 'publication':pub.bib['title']}, ignore_index=True)
    return(data)

def Crawler():
    df = pd.read_excel("./CoMSEf Member List 2020_Feb_11_20.xls")
    professor =  df[df['Mem Type'] == 'Professional']

    prof_num = 0
    data = pd.DataFrame(columns=['name','publication'])
    for idx,row in professor.iterrows():
        first_name = row['FirstName']
        middle_name = row['MiddleName']
        last_name = row['LastName']
        name = ''

        if isinstance(first_name,str):
            name +=  first_name
        if isinstance(middle_name,str):
            name +=' ' +  middle_name
        if isinstance(last_name,str):
            name +=' ' +  last_name
        print('Searching '+name+'...')

        try:
            data = data.append(search_name(name),ignore_index=True)
            print("Found!")
            prof_num += 1

        except:
            print("Not found!")
            
    print("Found ",prof_num)
    print("Saving dataframe....")
    data.to_excel('Data.xls',index=False)
    return(data)
    
def tweet(api,row,url):
    api.update_status(row['publication']+' '+url)
    print('twitter sent')

def FindNewPaper():
    old_data = pd.read_excel("./Data.xls")
    print(old_data.shape)
    data = Crawler()
    print(data.shape)
    new_data = pd.merge(data, old_data, on=['name','publication'], how="outer", indicator=True
              ).query('_merge=="left_only"')
    return(new_data)
#     display(new_data)
#     if new_data.shape[0]>1:
#         api = initAPI()
#         for idx,row in new_data.iterrows():
#             url = None
#             tweet(api,row,url)
            

In [25]:
new_data = FindNewPaper()

(30646, 2)
Searching Cameron Abrams...
Found!
Searching David Gregory Abrecht...
Not found!
Searching Irji Amrullah Adam...
Not found!
Searching Paul E Adamson...
Not found!
Searching Ravindra Aglave...
Found!
Searching Mohammed Almutawa...
Not found!
Searching Dilnoza Amirkulova...
Found!
Searching Brian Anderson...
Found!
Searching Stefan Andersson...
Found!
Searching Juan C Araque...
Found!
Searching Henry S Ashbaugh...
Found!
Searching Alireza Asiaee...
Found!
Searching Peng Bai...
Found!
Searching Perla B Balbuena...
Not found!
Searching Billy B Bardin...
Not found!
Searching Alcindor Bonamy...
Not found!
Searching James E Boone...
Not found!
Searching Joseph Brian...
Found!
Searching Linda Broadbelt...
Found!
Searching Brent Broehm...
Not found!
Searching David A. Bruce...
Found!
Searching David Cantu...
Found!
Searching James R Chelikowsky...
Found!
Searching Yongchul Chung...
Found!
Searching Paulette Clancy...
Not found!
Searching Coray Colina...
Not found!
Searching Yamil Col

In [26]:
new_dataata

,name,publication,_merge
254,Ravindra Aglave,Impact of Contact Scaling and Drag Calculation...,left_only
6334,Yamil Colón,Self-assembly of metal-organic frameworks,left_only
16852,John Kitchin,"Acceleration of catalyst discovery with easy, ...",left_only
22579,Erik Santiso,Exploring the physicochemical and morphologica...,left_only
30028,Lourdes Vega,Quantifying the Effect of Polarity on the Beha...,left_only
30084,Lourdes Vega,Quantifying the Effect of Polarity on the Beha...,left_only
32073,Xiao Cheng Zeng,Co-mixing hydrogen and methane may double the ...,left_only


In [45]:
if new_data.shape[0]>1:
    api = initAPI()
    for idx,row in new_data.iterrows():
        try:
            search_pub = scholarly.search_pubs(row['publication'])
            time.sleep(10)
            pub = next(search_pub)
            url = pub.bib['url']
        except:
            url = None
            print('Can not find url')
        
        if url != None:
            try:
                tweet(api,row,url)
            except:
                print('Can not tweet')
        except:
            print('Can not find url')
        time.sleep(60)


Authentication OK
Can not find url
Can not tweet
Can not find url
Can not tweet
Can not find url
Can not tweet
Can not find url
Can not tweet
Can not find url
Can not tweet
Can not find url
Can not tweet
Can not find url
Can not tweet


In [52]:
if new_data.shape[0]>1:
    api = initAPI()
    for idx,row in new_data.iterrows():
        print(row['name'],' ', row['publication'])

Authentication OK
Ravindra Aglave   Impact of Contact Scaling and Drag Calculation on the Accuracy of Coarse‐Grained Discrete Element Method
Yamil Colón   Self-assembly of metal-organic frameworks
John Kitchin   Acceleration of catalyst discovery with easy, fast, and reproducible computational alchemy
Erik Santiso   Exploring the physicochemical and morphological properties of peptide‐hybridized dendrimers (DendriPeps) and their aggregates
Lourdes Vega   Quantifying the Effect of Polarity on the Behavior of Mixtures of n ‐Alkanes with Dipolar Solvents using Polar soft‐SAFT
Lourdes Vega   Quantifying the Effect of Polarity on the Behavior of Mixtures of n‐Alkanes with Dipolar Solvents using Polar soft‐SAFT
Xiao Cheng Zeng   Co-mixing hydrogen and methane may double the energy storage capacity (vol 6, pg 8916, 2018)


In [ ]:
for status in api.home_timeline():
    print(status.id)
    try:
        api.destroy_status(status.id)
    except:
        print('failed')

In [ ]:
data = search_name('Coco Mao')

In [ ]:
data

In [ ]:
search_query = scholarly.search_author('Coco Mao')
author = next(search_query).fill()
pub = author.publications[0]
print(pub)

In [53]:
search_pub = scholarly.search_pubs('Molecular driving forces in peptide adsorption to metal oxide surfaces')

Exception: Cannot fetch the page from Google Scholar.

In [ ]:
pub = next(search_pub)

In [35]:
pub.bib['url']

'https://pubs.acs.org/doi/abs/10.1021/acs.langmuir.8b01392'

In [ ]:
search_query = scholarly.search_keyword('Molecular')

In [ ]:
query = next(search_query)

In [ ]:
query